In [211]:
from langgraph.graph import StateGraph, END
import random
from typing import TypedDict, List

In [212]:
class AgentState(TypedDict):
    player_name: str
    guesses: List[int]
    attempts: int
    lower_bound: int
    upper_bound: int
    number: int
    status: bool

In [213]:
import turtle
from langsmith.schemas import AttachmentsOperations


def setup_node(state: AgentState) -> AgentState:
    state['number'] = random.randint(state['lower_bound'], state['upper_bound'])
    state['status'] = False
    return state

def guess_node(state: AgentState) -> AgentState:
    state['guesses'].append(random.randint(state['lower_bound'], state['upper_bound']))
    return state

def hint_node(state: AgentState) -> AgentState:
    if state['guesses'][-1] > state['number']:
        print('Bro, too large!')
        state['upper_bound'] = state['guesses'][-1] - 1

    elif state['guesses'][-1] < state['number']:
        print('Bro, too small!')
        state['lower_bound'] = state['guesses'][-1] + 1
    
    else:
        print('correct!')
        state['status'] = True
    
    state['attempts'] += 1
    return state

def loop(state: AgentState) -> AgentState:
    if state['status'] == False and state['attempts'] < 7: # use 'and' instead of '&' next time
        return 'continue'
    else:
        return 'end'


In [214]:
graph = StateGraph(AgentState)

graph.add_node('setup', setup_node)
graph.add_node('guess', guess_node)
graph.add_node('hint', hint_node)
graph.add_edge('setup', 'guess')
graph.add_edge('guess', 'hint')

graph.add_conditional_edges(
    'hint',
    loop,
    {
        'continue': 'guess',
        'end': END
    }
)

graph.set_entry_point('setup')

app = graph.compile()

In [229]:
app.invoke({'player_name': 'student', 'guesses': [], 'attempts': 0, 'lower_bound': 1, 'upper_bound': 20})

Bro, too small!
Bro, too large!
Bro, too large!
Bro, too small!
Bro, too large!
Bro, too small!
Bro, too small!


{'player_name': 'student',
 'guesses': [3, 15, 14, 6, 13, 8, 11],
 'attempts': 7,
 'lower_bound': 12,
 'upper_bound': 12,
 'number': 12,
 'status': False}